In [ ]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from os.path import join as opj
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pylab
plt.rcParams['figure.figsize'] = 10, 10

In [ ]:
#Import Keras.
from matplotlib import pyplot
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from keras.layers import GlobalMaxPooling2D,GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam
from keras.optimizers import rmsprop
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping,ReduceLROnPlateau

from keras.datasets import cifar10
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception
from keras.applications.mobilenet import MobileNet
from keras.applications.vgg19 import VGG19
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.applications.xception import Xception

#Data Aug for multi-input
from keras.preprocessing.image import ImageDataGenerator


In [ ]:
import pandas as pd 
import numpy as np 
import cv2 # Used to manipulated the images 
np.random.seed(1337) # The seed I used - pick your own or comment out for a random seed. A constant seed allows for better comparisons though

In [ ]:
input_path = "F:/Kaggle/Statoil/inputs/"
batch_size = 32

In [ ]:
train = pd.read_json(input_path+"train.json")
target_train=train['is_iceberg']
test = pd.read_json(input_path+"test.json")

In [ ]:
def iso(arr):
    p = np.reshape(np.array(arr), [75,75]) >(np.mean(np.array(arr))+2*np.std(np.array(arr)))
    return p * np.reshape(np.array(arr), [75,75])
def size(arr):     
    return float(np.sum(arr<-5))/(75*75)

In [ ]:
### feat engg
train['iso1'] = train.iloc[:, 0].apply(iso)
train['iso2'] = train.iloc[:, 1].apply(iso)
test['iso1'] = test.iloc[:, 0].apply(iso)
test['iso2'] = test.iloc[:, 1].apply(iso)

# Feature engineering s1 s2 and size.
train['s1'] = train.iloc[:,5].apply(size)
train['s2'] = train.iloc[:,6].apply(size)
test['s1'] = test['iso1'].apply(size)
test['s2'] = test['iso2'].apply(size)

train['band_1'] = train['band_1'].apply(lambda x: np.array(x).reshape(75, 75))
train['band_2'] = train['band_2'].apply(lambda x: np.array(x).reshape(75, 75))
test['band_1'] = test['band_1'].apply(lambda x: np.array(x).reshape(75, 75))
test['band_2'] = test['band_2'].apply(lambda x: np.array(x).reshape(75, 75))

train['inc_angle'] = pd.to_numeric(train['inc_angle'], errors='coerce')
test['inc_angle'] = pd.to_numeric(test['inc_angle'], errors='coerce')


#####process test set!
band_1_test = np.concatenate([im for im in test['band_1']]).reshape(-1, 75, 75)
band_2_test = np.concatenate([im for im in test['band_2']]).reshape(-1, 75, 75)
full_img_test = np.stack([band_1_test, band_2_test,(band_1_test+band_2_test)/2], axis=1)
angle_test=test['inc_angle']
size_test=test['s1']
test['is_iceberg']=0

In [ ]:
target_train=train['is_iceberg']
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
train['inc_angle']=pd.to_numeric(train['inc_angle'], errors='coerce')#We have only 133 NAs.
train['inc_angle']=train['inc_angle'].fillna(method='pad')
X_angle=train['inc_angle']
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
X_test_angle=test['inc_angle']

In [ ]:
### concat angle and size
Tr_s1 = train['s1']
Tr_s2 = train['s2']
Te_s1 = test['s1']
Te_s2 = test['s2']

In [ ]:
def get_scaled_imgs(df):
    imgs = []
    
    for i, row in df.iterrows():
        #make 75x75 image
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = band_1 + band_2 # plus since log(x*y) = log(x) + log(y)
        
        # Rescale
        a = (band_1 - band_1.mean()) / (band_1.max() - band_1.min())
        b = (band_2 - band_2.mean()) / (band_2.max() - band_2.min())
        c = (band_3 - band_3.mean()) / (band_3.max() - band_3.min())

        imgs.append(np.dstack((a, b, c)))

    return np.array(imgs)

In [ ]:
X_train = get_scaled_imgs(train)
X_test = get_scaled_imgs(test)

In [ ]:
print("shape of train: {} and test: {}" .format(X_train.shape,X_test.shape))

In [ ]:
from clr_callback import *

In [ ]:
#### updated to include size and other variables

# Define the image transformations here
gen = ImageDataGenerator(horizontal_flip = True,
                         vertical_flip = True,
                         width_shift_range = 0.,
                         height_shift_range = 0.,
                         channel_shift_range=0,
                         zoom_range = 0.2,
                         rotation_range = 10)

# Here is the function that merges our two generators
# We use the exact same generator with the same random seed for both the y and angle arrays

def gen_flow_for_two_inputs(X1, X2, s1,s2,y):
    genX1 = gen.flow(X1,y,  batch_size=batch_size,seed=55)
    genX2 = gen.flow(X1,X2, batch_size=batch_size,seed=55)
    genX3 = gen.flow(X1,s1, batch_size=batch_size,seed=55)
    genX4 = gen.flow(X1,s2, batch_size=batch_size,seed=55)

    while True:
            X1i = genX1.next()
            X2i = genX2.next()
            X3i = genX3.next()
            X4i = genX4.next()
            
            #Assert arrays are equal - this was for peace of mind, but slows down training
            #np.testing.assert_array_equal(X1i[0],X2i[0])
            yield [X1i[0], (np.vstack((X2i[1],X3i[1],X4i[1])).T)], X1i[1]

# Finally create generator
# Finally create generator
def get_callbacks(filepath, patience=2):
    #es = EarlyStopping('val_loss', patience=10, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    #reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,patience=5, min_lr=0.01)
    #clr_triangular = CyclicLR(mode='triangular')
    clr_Cyclic = CyclicLR(mode='exp_range', gamma=0.99994)
    return [msave,clr_Cyclic]

In [ ]:
def getXceptionAngleModel():
    input_2 = Input(shape=[3,], name="angle")
    angle_layer = Dense(1,activation='relu')(input_2)
    base_model = Xception(weights='imagenet', include_top=False, pooling='avg')
    x = base_model.output
    

    #x = GlobalMaxPooling2D()(x)
    merge_one = concatenate([x, angle_layer])
    merge_one = Dense(216, activation='relu', name='fc2')(merge_one)
    merge_one = Dropout(0.3)(merge_one)
    merge_one = Dense(64, activation='relu', name='fc3')(merge_one)
    merge_one = Dropout(0.3)(merge_one)
    
    predictions = Dense(1, activation='sigmoid')(merge_one)
    
    model = Model(input=[base_model.input, input_2], output=predictions)
    
    sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])
    return model

In [ ]:
#Using K-fold Cross Validation with Data Augmentation.
def myAngleCV(X_train, X_angle,Tr_s1,Tr_s2, X_test):
    K=2
    folds = list(StratifiedKFold(n_splits=K, shuffle=True, random_state=16).split(X_train, target_train))
    y_test_pred_log = 0
    y_train_pred_log=0
    y_valid_pred_log = 0.0*target_train
    for j, (train_idx, test_idx) in enumerate(folds):
        print('\n===================FOLD=',j)
        X_train_cv = X_train[train_idx]
        y_train_cv = target_train[train_idx]
        X_holdout = X_train[test_idx]
        Y_holdout= target_train[test_idx]
        
        #Angle +size
        X_angle_cv=X_angle[train_idx]
        X_angle_hold=X_angle[test_idx]
        
        #size
        X_size1_cv = Tr_s1[train_idx]
        X_size2_cv = Tr_s2[train_idx]
        X_size1_hold = Tr_s1[test_idx]
        X_size2_hold = Tr_s2[test_idx]

        #define file path and get callbacks
        file_path = "%s_aug3_model_weights.hdf5"%j
        callbacks = get_callbacks(filepath=file_path, patience=5)
        gen_flow = gen_flow_for_two_inputs(X_train_cv, X_angle_cv,X_size1_cv,X_size2_cv, y_train_cv)
        galaxyModel= getXceptionAngleModel()
        galaxyModel.fit_generator(
                gen_flow,
                steps_per_epoch=24,
                epochs=100,
                shuffle=True,
                verbose=1,
                validation_data=([X_holdout,np.vstack((X_angle_hold,X_size1_hold,X_size2_hold)).T], Y_holdout),
                callbacks=callbacks)
        

        #Getting the Best Model
        galaxyModel.load_weights(filepath=file_path)
        #Getting Training Score
        score = galaxyModel.evaluate([X_train_cv,np.vstack((X_angle_cv,X_size1_cv,X_size2_cv)).T], y_train_cv, verbose=0)
        print('Train loss:', score[0])
        print('Train accuracy:', score[1])
        #Getting Test Score
        score = galaxyModel.evaluate([X_holdout,np.vstack((X_angle_hold,X_size1_hold,X_size2_hold)).T], Y_holdout, verbose=0)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])

        #Getting validation Score.
        pred_valid=galaxyModel.predict([X_holdout,np.vstack((X_angle_hold,X_size1_hold,X_size2_hold)).T])
        y_valid_pred_log[test_idx] = pred_valid.reshape(pred_valid.shape[0])

        #Getting Test Scores
        temp_test=galaxyModel.predict([X_test, np.vstack((X_test_angle,Te_s1,Te_s2)).T])
        y_test_pred_log+=temp_test.reshape(temp_test.shape[0])

        #Getting Train Scores
        temp_train=galaxyModel.predict([X_train, np.vstack((X_angle,Tr_s1,Tr_s2)).T])
        y_train_pred_log+=temp_train.reshape(temp_train.shape[0])

    y_test_pred_log=y_test_pred_log/K
    y_train_pred_log=y_train_pred_log/K

    print('\n Train Log Loss Validation= ',log_loss(target_train, y_train_pred_log))
    print(' Test Log Loss Validation= ',log_loss(target_train, y_valid_pred_log))
    return y_test_pred_log

In [ ]:
batch_size = 32
preds=myAngleCV(X_train, X_angle, Tr_s1,Tr_s2,X_test)

### Pseudo labels

In [ ]:
### choose i to maximize best choices for pseudo labels based on prob output
i = 0.047
idx_pred_1 = (np.where(preds>0.95+i))
idx_pred_0 = (np.where(preds<0.05-i))

In [ ]:
### add pseudolabels data to original train 
add_test = X_test[idx_pred_1]
add_angle = X_test_angle.iloc[idx_pred_1]
add_test_s1 = Te_s1.iloc[idx_pred_1]
add_test_s2 = Te_s2.iloc[idx_pred_1]

add_test1=X_test[idx_pred_0]
add_angle1 = X_test_angle.iloc[idx_pred_0]
add_test1_s1 = Te_s1.iloc[idx_pred_0]
add_test1_s2 = Te_s2.iloc[idx_pred_0]


additinal_test = np.concatenate((add_test,add_test1),axis=0)
additional_angle = np.concatenate((add_angle,add_angle1),axis=0)
additional_s1 = np.concatenate((add_test_s1,add_test1_s1),axis=0)
additional_s2 = np.concatenate((add_test_s2,add_test1_s2),axis=0)

additional_train = np.concatenate((X_train,additinal_test),axis=0)
additional_angle = np.concatenate((X_angle,additional_angle),axis=0)
additional_s1_tr = np.concatenate((Tr_s1,additional_s1),axis=0)
additional_s2_tr = np.concatenate((Tr_s2,additional_s2),axis=0)


Y_temp1 = [1]*add_test.shape[0]
Y_temp0 = [0]*add_test1.shape[0]

Ynew = np.concatenate((Y_temp1,Y_temp0),axis=0)
additionalY = np.concatenate((target_train,Ynew),axis=0)

In [ ]:
additionalY.shape[0]-target_train.shape[0]

In [ ]:
#Using K-fold Cross Validation with Data Augmentation.
def myAngleCV_pseud(additional_train, additional_angle,additional_s1_tr,additional_s2_tr, X_test):
    K=5
    folds = list(StratifiedKFold(n_splits=K, shuffle=True, random_state=16).split(additional_train, additionalY))
    y_test_pred_log = 0
    y_train_pred_log=0
    y_valid_pred_log = 0.0*additionalY
    for j, (train_idx, test_idx) in enumerate(folds):
        print('\n===================FOLD=',j)
        X_train_cv = additional_train[train_idx]
        y_train_cv = additionalY[train_idx]
        X_holdout = additional_train[test_idx]
        Y_holdout= additionalY[test_idx]
        
        #Angle +size
        X_angle_cv = additional_angle[train_idx]
        X_angle_hold = additional_angle[test_idx]
        
        #size
        X_size1_cv = additional_s1_tr[train_idx]
        X_size2_cv = additional_s2_tr[train_idx]
        X_size1_hold = additional_s1_tr[test_idx]
        X_size2_hold = additional_s2_tr[test_idx]

        #define file path and get callbacks
        file_path = "%s_aug4_model_weights.hdf5"%j
        callbacks = get_callbacks(filepath=file_path, patience=5)
        gen_flow = gen_flow_for_two_inputs(X_train_cv, X_angle_cv,X_size1_cv,X_size2_cv, y_train_cv)
        galaxyModel= getXceptionAngleModel()
        galaxyModel.fit_generator(
                gen_flow,
                steps_per_epoch=24,
                epochs=25,
                shuffle=True,
                verbose=1,
                validation_data=([X_holdout,np.vstack((X_angle_hold,X_size1_hold,X_size2_hold)).T], Y_holdout),
                callbacks=callbacks)
        

        #Getting the Best Model
        galaxyModel.load_weights(filepath=file_path)
        #Getting Training Score
        score = galaxyModel.evaluate([X_train_cv,np.vstack((X_angle_cv,X_size1_cv,X_size2_cv)).T], y_train_cv, verbose=0)
        print('Train loss:', score[0])
        print('Train accuracy:', score[1])
        #Getting Test Score
        score = galaxyModel.evaluate([X_holdout,np.vstack((X_angle_hold,X_size1_hold,X_size2_hold)).T], Y_holdout, verbose=0)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])

        #Getting validation Score.
        pred_valid=galaxyModel.predict([X_holdout,np.vstack((X_angle_hold,X_size1_hold,X_size2_hold)).T])
        y_valid_pred_log[test_idx] = pred_valid.reshape(pred_valid.shape[0])

        #Getting Test Scores
        temp_test=galaxyModel.predict([X_test, np.vstack((X_test_angle,Te_s1,Te_s2)).T])
        y_test_pred_log+=temp_test.reshape(temp_test.shape[0])

        #Getting Train Scores
        temp_train=galaxyModel.predict([additional_train, np.vstack((additional_angle,additional_s1_tr,additional_s2_tr)).T])
        y_train_pred_log+=temp_train.reshape(temp_train.shape[0])

    y_test_pred_log=y_test_pred_log/K
    y_train_pred_log=y_train_pred_log/K

    print('\n Train Log Loss Validation= ',log_loss(additionalY, y_train_pred_log))
    print(' Test Log Loss Validation= ',log_loss(additionalY, y_valid_pred_log))
    return y_test_pred_log

In [ ]:
batch_size = 32
predicition_pesudo=myAngleCV_pseud(additional_train, additional_angle,additional_s1_tr,additional_s2_tr, X_test)

In [ ]:
#Submission
import datetime as dt

submission = pd.DataFrame()
submission['id']=test['id']
submission['is_iceberg']=predicition_pesudo
submission.to_csv('F:/Kaggle/Statoil/submit/sub5cv_xception_pseudolabels_featengg'+dt.datetime.today().strftime("%d%m%Y%H%M")+'.csv', index=False)